In [ ]:
import sys
import argparse
import glob
import cv2
import os
import numpy as np
import torch
import time
from tqdm import tqdm
from pathlib import Path

from droid_slam.droid import Droid
import droid_backends

from lac.utils.plotting import plot_path_3d, plot_3d_points, plot_poses
from lac.util import load_data
from lac.params import LAC_BASE_PATH

%load_ext autoreload
%autoreload 2

# Load data


In [ ]:
def image_stream(datapath, image_size=[360, 640], intrinsics_vec=[914.0152, 914.0152, 640.0, 360.0], stereo=False):
    """image generator"""

    # read all png images in folder
    ht0, wd0 = [720, 1280]
    images_left = sorted(glob.glob(os.path.join(datapath, "FrontLeft/*.png")))
    images_right = sorted(glob.glob(os.path.join(datapath, "FrontRight/*.png")))

    data = []
    for t in range(len(images_left)):
        images = [cv2.resize(cv2.imread(images_left[t]), (image_size[1], image_size[0]))]
        if stereo:
            images += [cv2.resize(cv2.imread(images_right[t]), (image_size[1], image_size[0]))]

        images = torch.from_numpy(np.stack(images, 0)).permute(0, 3, 1, 2)
        intrinsics = 0.5 * torch.as_tensor(intrinsics_vec)

        data.append((t, images, intrinsics))

    return data

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--datapath", default="data/LAC")
parser.add_argument("--weights", default="/home/lac/opt/DROID-SLAM/droid.pth")
parser.add_argument("--buffer", type=int, default=1000)
parser.add_argument("--image_size", default=[360, 640])
parser.add_argument("--stereo", action="store_true")
parser.add_argument("--disable_vis", action="store_true")
parser.add_argument("--plot_curve", action="store_true")
parser.add_argument("--id", type=int, default=-1)

parser.add_argument("--beta", type=float, default=0.3)
parser.add_argument("--filter_thresh", type=float, default=2.4)
parser.add_argument("--warmup", type=int, default=12)
parser.add_argument("--keyframe_thresh", type=float, default=3.5)
parser.add_argument("--frontend_thresh", type=float, default=15)
parser.add_argument("--frontend_window", type=int, default=20)
parser.add_argument("--frontend_radius", type=int, default=1)
parser.add_argument("--frontend_nms", type=int, default=1)

parser.add_argument("--backend_thresh", type=float, default=20.0)
parser.add_argument("--backend_radius", type=int, default=2)
parser.add_argument("--backend_nms", type=int, default=3)
parser.add_argument("--upsample", action="store_true")

In [ ]:
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/stereo_side_lights1.0_map1_preset1"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
args = parser.parse_args(
    [
        "--datapath",
        str(data_path),
        "--disable_vis",
        "--stereo",
    ]
)

In [ ]:
stream = image_stream(args.datapath, stereo=args.stereo)

# Tracking


In [ ]:
BA_RATE = 1000
START_FRAME = 0

torch.cuda.empty_cache()
droid = Droid(args)

start_time = time.time()

for tstamp, image, intrinsics in tqdm(stream):
    droid.track(tstamp, image, intrinsics=intrinsics)

    # if tstamp % BA_RATE == 0:
    #     droid.backend(7)

print("Tracking ran {} frames in {} seconds".format(len(stream), time.time() - start_time))

In [ ]:
traj_est = droid.terminate(stream)

In [ ]:
droid.backend(12)

In [ ]:
camera_trajectory = droid.traj_filler(stream)
camera_trajectory = camera_trajectory.inv().data.cpu().numpy()

In [ ]:
from scipy.spatial.transform import Rotation


camera_poses = []

for vec in camera_trajectory:
    t = vec[:3]
    t = np.array([t[2], -t[0], -t[1]])
    q = vec[3:]
    R = Rotation.from_quat(q).as_matrix()
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    camera_poses.append(T)

In [ ]:
align_transform = initial_pose @ np.linalg.inv(camera_poses[50])

aligned_poses = [align_transform @ pose for pose in camera_poses[50:]]

In [ ]:
fig = plot_poses(poses, no_axes=True, name="Ground Truth", color="black")
fig = plot_poses(aligned_poses, fig=fig, no_axes=True, name="Droid")
fig.update_layout(width=1600, height=900, scene_aspectmode="data")
fig.show()

# Reconstruction


In [ ]:
from lietorch import SE3

In [ ]:
poses = droid.video.poses
disps = droid.video.disps


points = droid_backends.iproj(SE3(poses).inv().data, disps, droid.video.intrinsics[0]).cpu()

In [ ]:
points.shape

In [ ]:
pts = points.reshape(-1, 3).cpu().numpy()

In [ ]:
pts.shape

In [ ]:
import open3d as o3d
import numpy as np

In [ ]:
t = droid.video.counter.value
tstamps = droid.video.tstamp[:t].cpu().numpy()
images = droid.video.images[:t].cpu().numpy()
disps = droid.video.disps_up[:t].cpu().numpy()
poses = droid.video.poses[:t].cpu().numpy()
intrinsics = droid.video.intrinsics[:t].cpu().numpy()

In [ ]:
# depth scale factor
depth_scale = 1 / 256

point_cloud = o3d.geometry.PointCloud()

u, v = np.meshgrid(range(images.shape[3]), range(images.shape[2]))
x = (u - intrinsics[0, 2]) * disps / intrinsics[0, 0] * depth_scale
y = (v - intrinsics[1, 2]) * disps / intrinsics[1, 1] * depth_scale
z = disps * depth_scale

points = np.vstack((x.flatten(), y.flatten(), z.flatten(), np.ones_like(x.flatten()))).T

colors = images[:, [2, 1, 0], :, :].transpose(0, 2, 3, 1).reshape(-1, 3) / 255

point_cloud.points = o3d.utility.Vector3dVector(points[:, :3])
point_cloud.colors = o3d.utility.Vector3dVector(colors)


for i in range(poses.shape[0]):
    # homogeneous transform matrix로 변환
    pose_matrix = np.eye(4)
    pose_matrix[:3, :3] = Rotation.from_quat(poses[i, 3:]).as_matrix()
    pose_matrix[:3, 3] = poses[i, :3]

    # point_cloud 객체의 위치 및 방향 설정
    point_cloud.transform(pose_matrix)


o3d.visualization.draw_geometries([point_cloud])

In [ ]:
point_cloud

In [ ]:
plot_3d_points(pts[::100])

# Inspect splat slam output


In [ ]:
data_path = Path(LAC_BASE_PATH) / "output/LocalizationAgent/map1_preset0_4m_spiral"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

output_path = Path("/home/lac/Splat-SLAM/output")
video = np.load(output_path / "video_final.npz")

In [ ]:
slam_poses = video["poses"]

In [ ]:
fig = plot_poses(poses, no_axes=True, name="Ground Truth", color="black")
fig = plot_poses(slam_poses, fig=fig, no_axes=True, name="Splat-SLAM")
fig.update_layout(width=1600, height=900, scene_aspectmode="data")
fig.show()